In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split


In [2]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Updated fully connected layer
        self.fc1 = nn.Linear(32 * 8 * 8, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [3]:
# Hyperparameters
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

# Load and preprocess the dataset (Example: CIFAR-10)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

# Model, loss function, and optimizer
model = SimpleCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Validation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# Save the model if needed
torch.save(model.state_dict(), 'simple_cnn_model.pth')

Files already downloaded and verified
Epoch [1/5], Step [100/625], Loss: 1.7866
Epoch [1/5], Step [200/625], Loss: 1.5818
Epoch [1/5], Step [300/625], Loss: 1.5121
Epoch [1/5], Step [400/625], Loss: 1.2802
Epoch [1/5], Step [500/625], Loss: 1.1516
Epoch [1/5], Step [600/625], Loss: 1.3297
Epoch [2/5], Step [100/625], Loss: 1.1623
Epoch [2/5], Step [200/625], Loss: 1.3948
Epoch [2/5], Step [300/625], Loss: 1.0689
Epoch [2/5], Step [400/625], Loss: 1.1275
Epoch [2/5], Step [500/625], Loss: 0.9540
Epoch [2/5], Step [600/625], Loss: 1.3432
Epoch [3/5], Step [100/625], Loss: 1.0444
Epoch [3/5], Step [200/625], Loss: 1.1343
Epoch [3/5], Step [300/625], Loss: 0.9485
Epoch [3/5], Step [400/625], Loss: 0.8465
Epoch [3/5], Step [500/625], Loss: 1.2964
Epoch [3/5], Step [600/625], Loss: 0.7923
Epoch [4/5], Step [100/625], Loss: 0.9833
Epoch [4/5], Step [200/625], Loss: 1.0928
Epoch [4/5], Step [300/625], Loss: 1.2521
Epoch [4/5], Step [400/625], Loss: 1.0199
Epoch [4/5], Step [500/625], Loss: 1.0